In [1]:
%load_ext autoreload
%autoreload 2
from micron2.data import pull_nuclei
import pandas as pd
import pytiff
import h5py
import os

cells = pd.read_csv('/mnt/linux-data/codex/preprocessed_data/201021_BreastFFPE_Final/201021_BreastFFPE_Final_3_cells.csv', 
                    index_col=0, header=0)
nuclei_img = '/mnt/linux-data/codex/preprocessed_data/201021_BreastFFPE_Final/201021_BreastFFPE_Final_2_nuclei.tif'

imagefs = !ls /mnt/linux-data/codex/preprocessed_data/201021_BreastFFPE_Final/images/*.tif
dapi_images = [f for f in imagefs if 'DAPI' in f]
non_dapi_images = [f for f in imagefs if 'DAPI' not in f]
non_dapi_images = [f for f in non_dapi_images if 'Blank' not in f]
non_dapi_images = [f for f in non_dapi_images if 'Empty' not in f]

channel_names = [os.path.basename(x) for x in non_dapi_images]
channel_names = [x.replace(f'.tif','') for x in channel_names]
channel_names = [x.split('_')[-2] for x in channel_names]
channel_names = ["DAPI"] + channel_names
print(len(channel_names))

image_paths = [dapi_images[0]] + non_dapi_images
print(len(image_paths))

pull_nuclei(cells, 
            image_paths, 
            out_file='/dev/shm/dataset_debug.hdf5', 
            nuclei_img=nuclei_img,
            size=64,
            min_area=100, 
            channel_names=channel_names,
            debug=True
           )

40
40
Creating hdf5 file at /dev/shm/dataset_debug.hdf5
Pulling 500 cells


/home/ing/devel/micron2/micron2/data/pull_nuclei.py:43: UserWarning: The argument 'neighbors' is deprecated and will be removed in scikit-image 0.18, use 'connectivity' instead. For neighbors=4, use connectivity=1
  labelimg, n_labels = label(img, neighbors=4, return_num=True)


In [2]:
!ls -lha *.hdf5

-rw-rw-r-- 1 ing ing 1.7G Nov 15 15:48 dataset.hdf5


In [3]:
with h5py.File('/dev/shm/dataset_debug.hdf5', 'r') as f:
    print(f.keys())
    print(f['cells'].keys())
    print(f['intensity'].keys())
    print(f['meta'].keys())

<KeysViewHDF5 ['cells', 'intensity', 'meta']>
<KeysViewHDF5 ['C1q', 'CD103', 'CD11c', 'CD134', 'CD138', 'CD20', 'CD31', 'CD3e', 'CD4', 'CD40', 'CD40LG', 'CD45', 'CD45RA', 'CD45RO', 'CD64', 'CD68', 'CD69', 'CD8', 'CD80', 'CD89', 'CXCL13', 'CXCR5', 'DAPI', 'FOXP3', 'GZMB', 'HLA-DR', 'IL7R', 'IgA', 'IgG', 'IgM', 'Ki-67', 'LAG3', 'OX40L', 'PD-1', 'PD-L1', 'PDGFRb', 'PNaD', 'PanCytoK', 'TIM3', 'aSMA']>
<KeysViewHDF5 ['C1q', 'CD103', 'CD11c', 'CD134', 'CD138', 'CD20', 'CD31', 'CD3e', 'CD4', 'CD40', 'CD40LG', 'CD45', 'CD45RA', 'CD45RO', 'CD64', 'CD68', 'CD69', 'CD8', 'CD80', 'CD89', 'CXCL13', 'CXCR5', 'DAPI', 'FOXP3', 'GZMB', 'HLA-DR', 'IL7R', 'IgA', 'IgG', 'IgM', 'Ki-67', 'LAG3', 'OX40L', 'PD-1', 'PD-L1', 'PDGFRb', 'PNaD', 'PanCytoK', 'TIM3', 'aSMA']>
<KeysViewHDF5 ['Cell_IDs', 'channel_names', 'coordinates', 'nuclear_masks']>


In [4]:
from micron2.data import load_as_anndata

In [8]:
adata = load_as_anndata('/dev/shm/dataset_debug.hdf5')
adata

AnnData object with n_obs × n_vars = 500 × 40

In [9]:
adata.write('f.h5ad')

In [7]:
!pwd

/home/ing/devel/micron2/notebooks/tests
